<a href="https://colab.research.google.com/github/alshemc/AndroidPacman/blob/master/radlogics_dataset_source_and_segments_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copy and run locally due to large dataset size.
All imported packages must be installed.

In [ ]:
!pip install SimpleITK

In [ ]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import glob as glob
import matplotlib.pyplot as plt

%matplotlib inline

Paths to features and labels, and csv file.


In [ ]:
segments_path = r'D:\manifest-1586193031612\Effusions' #/LUNG1-001/LUNG1-001_effusion_first_reviewer.nii.gz
sources_path = r'D:\manifest-1586193031612\NSCLC-Radiomics' #/LUNG1-001/09-18-2008-StudyID-NA-69331/0.000000-NA-82046
csv_path = 'D:/manifest-1586193031612/_docs/Thoracic and Pleural Effusion Segmentations April 2020.csv'

In [ ]:
df = pd.read_csv(csv_path)
df.set_index("PatientID", inplace=True)

# Сбор информации об особенностях исходных данных

In [ ]:
#создание dataframe и проверка количества slices.
sources_df = pd.DataFrame(columns=["PatientID","Feature.Slices"])
sources_df.set_index("PatientID", inplace=True)

In [ ]:
#заполнение датафрейма актуальными данными из каталога

sources_folders = sorted(glob.glob(f"{sources_path}/*"))

for folder in sources_folders:
    id = folder.split("\\")[-1]   # для windows, для linux должен быть другой разделитель "/"
    #print (id, " - ", folder)
    dicom_folder = glob.glob(f'{glob.glob(f"{folder}/*")[0]}/*')[0]
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(dicom_folder)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    slices = image.GetSize()[2]
    sources_df.loc[id, "Feature.Slices"] = int(slices)

LUNG1-001  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-001
LUNG1-002  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-002
LUNG1-004  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-004
LUNG1-005  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-005
LUNG1-006  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-006
LUNG1-007  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-007
LUNG1-008  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-008
LUNG1-009  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-009
LUNG1-010  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-010
LUNG1-011  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-011
LUNG1-012  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-012
LUNG1-013  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-013
LUNG1-015  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-015
LUNG1-016  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-016
LUNG1-017  -  D:\manifest-1586193031612\NSCLC-Radiomics\LUNG1-017
LUNG1-018 

In [ ]:
sources_merged_df = pd.merge(df.loc[:,'Dim.z'], sources_df.astype(np.int16), left_index=True, right_index=True, how='left')
sources_merged_df.head(5)

,Dim.z,Feature.Slices
PatientID,,
LUNG1-001,134.0,134.0
LUNG1-002,111.0,111.0
LUNG1-003,NaN,NaN
LUNG1-004,114.0,114.0
LUNG1-005,91.0,91.0


In [ ]:
#проверка разницы между информацией в csv и актуальными файлами
sources_merged_df['Feature.Slices'].equals(sources_merged_df['Dim.z'])

False

In [ ]:
sources_merged_df.loc[(sources_merged_df['Dim.z'] != sources_merged_df['Feature.Slices'])]

,Dim.z,Feature.Slices
PatientID,,
LUNG1-003,NaN,NaN
LUNG1-014,NaN,NaN
LUNG1-021,NaN,NaN
LUNG1-031,NaN,NaN
LUNG1-058,NaN,NaN
LUNG1-061,NaN,NaN
LUNG1-069,NaN,NaN
LUNG1-074,NaN,NaN
LUNG1-083,113.0,NaN


In [ ]:
#проверка количества слайсов в csv и текущем каталоге на равернство
clean_sources_merged_df = sources_merged_df.dropna(axis=0, subset=["Feature.Slices","Dim.z"]).astype(np.int16)
clean_sources_merged_df['Feature.Slices'].equals(clean_sources_merged_df['Dim.z'])

True

In [ ]:
#пометка объектов на корректность есди существуют как csv-данные, так и сами файлы
sources_merged_df['Valid.Feature'] = np.where((sources_merged_df.isna()['Dim.z'] == False) & (sources_merged_df.isna()['Feature.Slices'] == False), 1, np.nan)
sources_merged_df.drop(columns=['Dim.z'], inplace=True)
sources_merged_df.head(15)

,Feature.Slices,Valid.Feature
PatientID,,
LUNG1-001,134.0,1.0
LUNG1-002,111.0,1.0
LUNG1-003,NaN,NaN
LUNG1-004,114.0,1.0
LUNG1-005,91.0,1.0
LUNG1-006,114.0,1.0
LUNG1-007,129.0,1.0
LUNG1-008,114.0,1.0
LUNG1-009,105.0,1.0


# Сбор информации о сегментах

## Создание датафреймов для сегментов

In [ ]:
#Список ранее исключенных меток, основанный на отчетах рентгенолога.
previously_excluded_segments = ["LUNG1-013", "LUNG1-170", "LUNG1-195", "LUNG1-205", "LUNG1-249", "LUNG1-253", "LUNG1-303", "LUNG1-320", "LUNG1-331", "LUNG1-336", "LUNG1-340", "LUNG1-348", "LUNG1-362", "LUNG1-381", "LUNG1-416", "LUNG1-418"]

In [ ]:
#Сжатие фрейма данных до строк с существующими событиями выпота
segments_df = df.loc[(df["Effusion.Event"] == 1.0)]
segments_df.head(5)

,Carcinoma.Laterality,GTV1,GTV2,GTV3,GTV4,GTV5,GTV6,Tumor.Location,Effusion.Event,Primary.Effusion.Reviewer,...,RO1-RO3.Thorax.DSC,Rad1-Rad3.Thorax.DSC,Rad1-Rad2.Thorax.DSC,RO2-Rad2.Thorax.DSC,Dim.x,Dim.y,Dim.z,Voxel.Space.x,Voxel.Space.y,Voxel.space.z
PatientID,,,,,,,,,,,,,,,,,,,,,
LUNG1-001,L,139.06,NaN,NaN,NaN,NaN,NaN,3.0,1.0,Rad4,...,NaN,NaN,NaN,NaN,512.0,512.0,134.0,0.976563,0.976563,3.0
LUNG1-002,R,340.30,NaN,NaN,NaN,NaN,NaN,3.0,1.0,Rad4,...,NaN,NaN,NaN,NaN,512.0,512.0,111.0,0.977000,0.977000,3.0
LUNG1-005,R,78.62,NaN,NaN,NaN,NaN,NaN,1.0,1.0,Rad4,...,NaN,NaN,NaN,NaN,512.0,512.0,91.0,0.977000,0.977000,3.0
LUNG1-008,R,37.48,NaN,NaN,NaN,NaN,NaN,3.0,1.0,Rad4,...,NaN,NaN,NaN,NaN,512.0,512.0,114.0,0.977000,0.977000,3.0
LUNG1-013,L,13.25,NaN,NaN,NaN,NaN,NaN,1.0,1.0,Rad4,...,NaN,NaN,NaN,NaN,512.0,512.0,134.0,0.976563,0.976563,3.0


In [ ]:
#1.Удаление столбцов, за исключением тех, которые содержат данные о рецензентах.
#2.Подсчет количества доступных отзывов (меток).
#3.Удаление начальных столбцов с данными рецензентов.

segments_df = segments_df.loc[:,["Effusion.Event", "Primary.Effusion.Reviewer", "Secondary.Effusion.Reviewer", "Tertiary.Effusion.Reviewer"]]
segments_df['Labels.Amount'] = 3 - segments_df.loc[:, ["Primary.Effusion.Reviewer", "Secondary.Effusion.Reviewer", "Tertiary.Effusion.Reviewer"]].isna().sum(axis=1)
segments_df.drop(["Effusion.Event", "Primary.Effusion.Reviewer", "Secondary.Effusion.Reviewer", "Tertiary.Effusion.Reviewer"],axis=1, inplace=True)

In [ ]:
#1.Создаем столбец для пометки уже исключенных меток начальным значением = 0.
#2.Помечать исключенные метки знаком 1.
segments_df.loc[:, "Previously.Excluded"] = 0
segments_df.loc[segments_df.index.isin(previously_excluded_segments), "Previously.Excluded"] = 1

segments_df.head(5)

,Labels.Amount,Previously.Excluded
PatientID,,
LUNG1-001,2,0
LUNG1-002,3,0
LUNG1-005,3,0
LUNG1-008,2,0
LUNG1-013,2,1


## Вспомогательные функции

In [ ]:
#Преобразование изображения в массив слайса (фрагмента/среза).
def get_image_array(path):
    nifti = sitk.ReadImage(path,  imageIO="NiftiImageIO")
    return sitk.GetArrayFromImage(nifti)

In [ ]:
#Получаем количество ненулевых вокселов (voxels) в каждом срезе.
def get_values_by_slices(arr):
    return np.array([np.sum(abs(i)) for i in arr.astype(bool)])

In [ ]:
# Подсчёт пробелов:
# - Установка границ изображения
# - Поиск нулей

def get_gaps(arr):
    image_boundaries = [np.flatnonzero(arr)[0], (np.flatnonzero(arr)[-1])]
    counter = {
        "indices": []
    }
    gaps = []

    for idx, item in enumerate(arr):
        if idx > image_boundaries[0] and idx < image_boundaries[1]:
            if item == 0:
                counter["indices"].append(idx+1)
            elif item != 0 and len(counter['indices']) > 0:
                 gaps.append(np.array(counter["indices"]))
                 counter["indices"] = []

    return gaps

In [ ]:
# Подсчет выбросов:
# 1.Установка границ изображения
# 2.Подсчет последующих непустых срезов
# 3.Сравнение количества последовательных непустых срезов с пороговым значением

def get_outliers(arr, threshold):
    """
        args:
            arr: 1D array
            threshold: int, maximum amount of slices treated as outliers
        return:
            array: [ndarray]
    """
    image_boundaries = [np.flatnonzero(arr)[0], (np.flatnonzero(arr)[-1])]
    outliers = []
    counter = {
        "indices": []
    }

    for idx, item in enumerate(arr):
        if idx >= image_boundaries[0] and idx <= image_boundaries[1]:
            if item != 0:
                counter["indices"].append(idx+1)
            elif item == 0:
                if len(counter['indices']) > 0 and len(counter['indices']) <= threshold:
                    outliers.append(np.array(counter['indices']))

                counter['indices'] = []

    if len(counter['indices']) > 0 and len(counter['indices']) <= threshold:
                    outliers.append(np.array(counter['indices']))

    return outliers

In [ ]:
# Получение данных о пробелах (gaps) и выбросах (outliers), количестве срезов (slices).
def check_segment(path):
    image_array = get_image_array(path)
    values_by_slices = get_values_by_slices(image_array)
    gaps = get_gaps(values_by_slices)
    outliers = get_outliers(values_by_slices, 3)
    return {
         "gaps": gaps,
         "outliers": outliers,
         "amount_of_slices": image_array.shape[0]
    }

In [ ]:
# Определение, является ли метка подозрительной, основываясь на наличии пробелов или выбросов.
def is_suspicious(label):
     return len(label["gaps"]) > 0 or len(label["outliers"]) > 0

In [ ]:
# Получение данных о текущем файле:
# 1.Рецензент
# 2.Идентификатор пациента

def get_path_data(path):
    path_arr = path.split('\\')       # для windows, для linux должен быть другой разделитель "/"
    #print(path_arr)
    patient_id = path_arr[3]          # индекс папки вложенности в с данными пациента
    reviewer = 0
    if 'first' in path:
        reviewer = "First.Reviewer."
    elif 'second' in path:
        reviewer = "Second.Reviewer."
    elif 'third' in path:
        reviewer = "Third.Reviewer."

    return {"patient_id": patient_id, "reviewer": reviewer}

In [ ]:
#ЗАДАЧА: Получение наиболее точной сегментации на основе: RO1-Rad4.Effusion.DSC, Rad3-Rad4.Effusion.DSC, RO1-Rad3.Effusion.DSC
# 1. Первый => Rad4
# 2. Второй => Rad3
# 3. Третий => RO1

def get_most_accurate_segment(id, clean_segments):
    reviewers = ['Rad4', 'Rad3', 'RO1']
    comparisons = ['RO1-Rad4.Effusion.DSC', 'Rad3-Rad4.Effusion.DSC', 'RO1-Rad3.Effusion.DSC']
    accurate_segment = None

    for l in clean_segments:
        index = l-1
        reviewer = reviewers[index]
        columns = list(filter(lambda x: reviewer in x, comparisons))
        #data = df.loc[id, ['RO1-Rad4.Effusion.DSC', 'Rad3-Rad4.Effusion.DSC', 'RO1-Rad3.Effusion.DSC']]

get_most_accurate_segment(id, [1,2])

In [ ]:
# Построение отчета о количестве срезов сегментов, пробелов и выбросов; количестве чистых сегментов
# 1.Создание датафрейма отчета
# 2.Контрольная метка (сегмент)
# 3.Добавление строки в отчет

def collect_segments_data(path):
    report_df = pd.DataFrame(columns=["PatientID", "Most.Accurate.Label","Valid.Label", "Clean.Reviews", "First.Reviewer.Slices", "First.Reviewer.Gaps", "First.Reviewer.Outliers", "Second.Reviewer.Slices", "Second.Reviewer.Gaps", "Second.Reviewer.Outliers", "Third.Reviewer.Slices", "Third.Reviewer.Gaps", "Third.Reviewer.Outliers"])
    report_df.set_index("PatientID")
    report_df.drop("PatientID", axis=1, inplace=True)

    folders = sorted(glob.glob(f"{path}/*"))

    for folder in folders:
        patient_id = get_path_data(folder)["patient_id"]
        files = sorted(glob.glob(f"{folder}/*"))
        clean_reviews = []
        for idx, file in enumerate(files):
            file_data = get_path_data(file)
            segment_data = check_segment(file)

            report_df.loc[patient_id, "Valid.Label"] = 0
            report_df.loc[file_data['patient_id'], f'{file_data["reviewer"]}Slices'] = int(segment_data["amount_of_slices"])
            if is_suspicious(segment_data):
                if len(segment_data["gaps"]) > 0:
                    report_df.loc[file_data['patient_id'], f'{file_data["reviewer"]}Gaps'] = segment_data['gaps']
                if len(segment_data["outliers"]) > 0:
                    report_df.loc[file_data['patient_id'], f'{file_data["reviewer"]}Outliers'] = segment_data['outliers']
            else:
                clean_reviews.append(idx+1)

        if len(clean_reviews) > 0:
                report_df.loc[patient_id, "Valid.Label"] = 1
                report_df.loc[patient_id, "Clean.Reviews"] = clean_reviews

    return report_df

## Построение финального отчёта

In [ ]:
# Построение отчёта на сегментах
segments_report = collect_segments_data(segments_path)

In [ ]:
# Объединение фреймов данных по объектам и сегментам (меткам).
final_segments_df = pd.merge(segments_df, segments_report, left_index=True, right_index=True, how='left')
sources_segments_df = pd.merge(sources_merged_df, final_segments_df, left_index=True, right_index=True, how='left')
sources_segments_df.head(5)

,Feature.Slices,Valid.Feature,Labels.Amount,Previously.Excluded,Most.Accurate.Label,Valid.Label,Clean.Reviews,First.Reviewer.Slices,First.Reviewer.Gaps,First.Reviewer.Outliers,Second.Reviewer.Slices,Second.Reviewer.Gaps,Second.Reviewer.Outliers,Third.Reviewer.Slices,Third.Reviewer.Gaps,Third.Reviewer.Outliers
PatientID,,,,,,,,,,,,,,,,
LUNG1-001,134.0,1.0,2.0,0.0,NaN,1,"[1, 2]",134,NaN,NaN,134,"[[81, 82, 83, 84, 85, 86, 87]]",NaN,NaN,NaN,NaN
LUNG1-002,111.0,1.0,3.0,0.0,NaN,1,"[2, 3]",111,"[[40, 41]]",NaN,111,NaN,NaN,111,NaN,NaN
LUNG1-003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LUNG1-004,114.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LUNG1-005,91.0,1.0,3.0,0.0,NaN,1,"[2, 3]",91,"[[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, ...","[[20, 21, 22]]",91,NaN,NaN,91,NaN,NaN


In [ ]:
# проверяем целостность по количеству срезов
clean_sources_segments_df = sources_segments_df.dropna(axis=0, subset=["Feature.Slices", "First.Reviewer.Slices"]).loc[:, ["Feature.Slices", "First.Reviewer.Slices", "Second.Reviewer.Slices", "Third.Reviewer.Slices"]]
clean_sources_segments_df.head(5)

,Feature.Slices,First.Reviewer.Slices,Second.Reviewer.Slices,Third.Reviewer.Slices
PatientID,,,,
LUNG1-001,134.0,134,134,NaN
LUNG1-002,111.0,111,111,111
LUNG1-005,91.0,91,91,91
LUNG1-008,114.0,114,114,NaN
LUNG1-013,134.0,134,134,NaN


In [ ]:
# Сравнение фрагментов по исходным объектам и первый обзор.
clean_sources_segments_df['Feature.Slices'].astype(np.int16).equals(clean_sources_segments_df['First.Reviewer.Slices'].astype(np.int16))

True

In [ ]:
# Сравнение фрагментов по исходным объектам и второй обзор.
clean_sources_segments_df.dropna(axis = 0, subset="Second.Reviewer.Slices")['Feature.Slices'].astype(np.int16).equals(clean_sources_segments_df.dropna(axis = 0, subset="Second.Reviewer.Slices")['Second.Reviewer.Slices'].astype(np.int16))

True

In [ ]:
# Сравнение фрагментов по исходным объектам и третий обзор.
clean_sources_segments_df.dropna(axis = 0, subset="Third.Reviewer.Slices")['Feature.Slices'].astype(np.int16).equals(clean_sources_segments_df.dropna(axis = 0, subset="Third.Reviewer.Slices")['Third.Reviewer.Slices'].astype(np.int16))

True

Merging all report to initial dataset csv file.

In [ ]:
# Объединение всего отчета в исходный csv-файл набора данных.

final_df = pd.merge(df, sources_segments_df, left_index=True, right_index=True, how='left')
final_df.reset_index(inplace=True)
final_df.head(10)

,PatientID,Carcinoma.Laterality,GTV1,GTV2,GTV3,GTV4,GTV5,GTV6,Tumor.Location,Effusion.Event,...,Clean.Reviews,First.Reviewer.Slices,First.Reviewer.Gaps,First.Reviewer.Outliers,Second.Reviewer.Slices,Second.Reviewer.Gaps,Second.Reviewer.Outliers,Third.Reviewer.Slices,Third.Reviewer.Gaps,Third.Reviewer.Outliers
0,LUNG1-001,L,139.06,NaN,NaN,NaN,NaN,NaN,3.0,1.0,...,"[1, 2]",134,NaN,NaN,134,"[[81, 82, 83, 84, 85, 86, 87]]",NaN,NaN,NaN,NaN
1,LUNG1-002,R,340.30,NaN,NaN,NaN,NaN,NaN,3.0,1.0,...,"[2, 3]",111,"[[40, 41]]",NaN,111,NaN,NaN,111,NaN,NaN
2,LUNG1-003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LUNG1-004,L,86.50,7.06,70.38,NaN,NaN,NaN,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LUNG1-005,R,78.62,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,"[2, 3]",91,"[[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, ...","[[20, 21, 22]]",91,NaN,NaN,91,NaN,NaN
5,LUNG1-006,L,75.90,NaN,NaN,NaN,NaN,NaN,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LUNG1-007,R,9.62,15.38,NaN,NaN,NaN,NaN,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LUNG1-008,R,37.48,NaN,NaN,NaN,NaN,NaN,3.0,1.0,...,[1],114,NaN,NaN,114,NaN,[[81]],NaN,NaN,NaN
8,LUNG1-009,R,91.85,32.48,37.24,83.87,NaN,NaN,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,LUNG1-010,R,15.50,5.72,NaN,NaN,NaN,NaN,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Saving to csv.

In [ ]:
final_df.to_csv('D:/manifest-1586193031612/_docs/dataset_sources_and_segments_report.csv', index=False)